In [0]:
# import pyspark
# import glow

In [0]:
# builder = pyspark.sql.SparkSession.builder.appName("GlowVCFExplore") \
#     .config("spark.jars.packages", "io.projectglow:glow-spark3_2.12:1.2.1") \
#     .config("spark.hadoop.io.compression.codecs", "io.projectglow.sql.util.BGZFCodec")

In [0]:
# spark = builder.getOrCreate()

In [0]:
# spark = glow.register(spark)

In [0]:
# spark

In [0]:
# Page Break

# Explore bcBio Germline VCF

In [0]:
# bcbio_germline_src = "./data/bcbio_giab_somatic/na12878-na24385-germline-hg38-truth.vcf.gz"
bcbio_germline_src = "dbfs:/tmp/dannywong/na12878-na24385-germline-hg38-truth.vcf.gz"

In [0]:
bcbio_germline_df = spark.read.format("vcf").load(bcbio_germline_src)

In [0]:
bcbio_germline_df.printSchema()

In [0]:
bcbio_germline_df.createOrReplaceTempView("bcbio_germline_table")

In [0]:
spark.sql("describe bcbio_germline_table").show(n=1000, truncate=True)

In [0]:
spark.sql("select count(1) as number_of_variants from bcbio_germline_table").show()

In [0]:
spark.sql("select distinct contigName from bcbio_germline_table order by contigName").show(46)

In [0]:
spark.sql("select contigName, start, end from bcbio_germline_table").show()

In [0]:
spark.sql("select contigName, count(end) as num_of_pos from bcbio_germline_table group by contigName order by num_of_pos desc").show(46)

In [0]:
spark.sql("select referenceAllele, alternateAlleles, array_size(alternateAlleles) from bcbio_germline_table").show()

In [0]:
spark.sql("select referenceAllele, alternateAlleles, count(*) as num_of_snps \
from bcbio_germline_table \
where \
    char_length(referenceAllele) = 1 and \
    array_size(alternateAlleles) = 1 and \
    char_length(alternateAlleles[0]) = 1 \
    group by referenceAllele, alternateAlleles \
    order by num_of_snps desc").show()

In [0]:
spark.sql("select referenceAllele, alternateAlleles, count(*) as num_of_deletions \
from bcbio_germline_table \
where \
    char_length(referenceAllele) > 1 and \
    array_size(alternateAlleles) = 1 and \
    char_length(alternateAlleles[0]) = 1 \
    group by referenceAllele, alternateAlleles \
    order by num_of_deletions desc").show()

In [0]:
spark.sql("select referenceAllele, alternateAlleles, count(*) as num_of_insertions \
from bcbio_germline_table \
where \
    char_length(referenceAllele) = 1 and \
    array_size(alternateAlleles) > 1 \
    group by referenceAllele, alternateAlleles \
    order by num_of_insertions desc").show()

In [0]:
spark.sql("select count(1) as `A>C` from bcbio_germline_table \
where \
    referenceAllele = 'A' and \
    alternateAlleles[0] = 'C' and \
    array_size(alternateAlleles) = 1").show()

In [0]:
spark.sql("select count(1) as `A>AT` from bcbio_germline_table \
where \
    referenceAllele = 'A' and \
    alternateAlleles[0] = 'AT' and \
    array_size(alternateAlleles) = 2").show()

In [0]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId, genotypes.alleleDepths \
from bcbio_germline_table").show(truncate=False)

In [0]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId from bcbio_germline_table \
where contigName = 'chr1' and end = 817186").show(truncate=False)

In [0]:
spark.sql("select contigName, start, end, referenceAllele, alternateAlleles, genotypes.sampleId[0] from bcbio_germline_table \
where genotypes.sampleId[0] = 'NA12878'").show(truncate=False)

In [0]:
# Page Break

# Stop Spark Session

In [0]:
# spark.stop()

In [0]:
# Continue to next notebook